## <center>Só Ninjas</center>

# <center><u>``Resumo do código``</u></center>

### <u>Código que pega nos ficheiros da Sonae e cria </u>

>### Entra: 
> - 1_Ninjas

>### Sai: 
> - Com Fotos
> - Sem Fotos

In [12]:
import pandas as pd
import numpy as np
import datetime
import json
from ipywidgets import interact

def escrever_excel(dfa, nome):
    dfa.to_excel(f'C:\\Users\\Chip7\\Desktop\\B&N\\2.Delta\\Sonae\\Testes\\{nome}.xlsx', index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(f'C:\\Users\\Chip7\\Desktop\\B&N\\2.Delta\\Sonae\\Testes\\{nome}.csv', index=False)
    
def escrever_txt(dfa, nome):
    dfa.to_csv(f'C:\\Users\\Chip7\\Desktop\\B&N\\2.Delta\\Sonae\\Testes\\{nome}.txt', sep='\t', index=False, header=False)
    
def ler_json(ficheiro):
    with open(f'C:\\Users\\Chip7\\Desktop\\B&N\\2.Delta\\Sonae\\Testes\\{ficheiro}.json', 'r') as file:
        mapa = json.load(file)
    return mapa

<a id="section0"></a>

# Índice

1. [Data Cleaning](#section1)<br>
2. [DataFusion Long](#section2)

<a id="section1"></a>
# <center>1. Data Cleaning</center>
[Voltar ao índice](#section0)<br>


- # `Dicionários super úteis`

In [13]:
''' Ler os dicionários previamente criados para poder usar os códigos de produto e de loja em vez dos nomes '''

mapa_produtos = ler_json("dicionário_produtos")
mapa_produtos = {int(key): value for key, value in mapa_produtos.items()}

mapa_produtos2 = ler_json("dicionário_produtos_23")
mapa_produtos2 = {int(key): value for key, value in mapa_produtos2.items()} 

mapa_lojas = ler_json("dicionário_lojas")
mapa_lojas = {int(key): value for key, value in mapa_lojas.items()}

- # `Ninjas`

> <font color=blue>Ler</font>
<a id="section1.1"></a>

In [14]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento

directória = "C:\\Users\\Chip7\\Desktop\\B&N\\2.Delta\\Sonae\\Diario\\"
num_bases = 1

prefixo = "1_Ninjas"
extensão = ".xlsx" #xlsb


dataframes = []

for i in range(1, num_bases + 1):
    file_path = f"{directória}{prefixo}{i}{extensão}"
    df = pd.read_excel(file_path)
    dataframes.append(df)
    
dfNinjas = pd.concat(dataframes, ignore_index=True)
dfNinjas = dfNinjas.rename(columns={'Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"STORE_NAME"})


# Quando tiver a porcaria dos EANS
#dfNinjas = dfNinjas.rename(columns=lambda x: mapa_produtos2.get(x, x))

"""Datetime"""

# se xlsx
dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], format='%d-%m-%Y') 

# se xlsb
#dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

produtos = dfNinjas.columns[3:].tolist()
dfNinjas = dfNinjas.dropna(how = "all")

CPU times: total: 328 ms
Wall time: 926 ms


> - Lojas, produtos e dicionário de lojas

In [15]:
lojas = dfNinjas.STORE_NAME.unique().tolist()
codlojas = dfNinjas.STORE.unique().tolist()
datas = dfNinjas.DATA.unique()

numprod = len(produtos)


ficheiroProdutos = pd.DataFrame(produtos, columns=["Produtos"])
ficheiroLojas = pd.DataFrame(lojas, columns=["Produtos"])
dicionarioLojas = dict(zip(dfNinjas['STORE'].unique(), dfNinjas['STORE_NAME'].unique()))

> - Número de produtos presentes

In [16]:
dfNinjas["Num_Produtos"] = dfNinjas[produtos].sum(axis=1)

> - Dia da Semana e Fim de Semana

In [17]:
dfNinjas['Dia_da_Semana'] = dfNinjas['DATA'].dt.day_name().map({'Friday': 'Sexta', 
                                                                'Saturday': 'Sábado', 
                                                                'Sunday': 'Domingo'})

dfNinjas['FDS'] = dfNinjas['DATA'].dt.isocalendar().week
dfNinjas['FDS'] = dfNinjas.groupby('FDS').ngroup() + 1

mapping = {1: "9-10 Setembro",
           2: "16-17 Setembro",
           3: "23-24 Setembro",
           4: "30-01 Setembro/Outubro"}
dfNinjas["Order_FDS"] = dfNinjas["FDS"]
dfNinjas["FDS"] = dfNinjas["FDS"].map(mapping)

In [26]:
[f"{index}: {column}" for index, column in enumerate(dfOrganizado.columns.tolist())]

['0: DATA',
 '1: STORE_NAME',
 '2: STORE',
 '3: Num_Produtos',
 '4: Dia_da_Semana',
 '5: FDS',
 '6: Order_FDS',
 '7: CAFFELATTE GO CHILL DELTA 230 ML',
 '8: CAPPUCCINO GO CHILL DELTA 230 ML',
 '9: DOUBLE ESPRESSO GO CHILL DELTA 230ML',
 '10: CARAMEL MACCHIATO GO CHILL DELTA 230ML',
 '11: LATTE MACCHIATO GO CHILL S/LAC 230ML']

In [27]:
dfNinjas = dfNinjas.iloc[:, np.r_[2,0:2,8:12,3:8]]

---

<a id="section2"></a>
# <center> 2. DataFusion Long</center>
[Voltar ao índice](#section0)

- #  `dfFinal1`

In [30]:
resto = dfNinjas.columns.tolist()[:7]
# Passar ficheiro ninjas para long
dfNinjasLong = dfNinjas.melt(id_vars=resto, value_vars=produtos, var_name='DESC_ARTIGO', value_name='NinjaInfo')


#Mergir
#dfMeio = pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE", "DESC_ARTIGO"]) 
#dfMeio = dfMeio.drop(columns=["Loja"])

#dfFinal1=dfMeio.copy()

In [31]:
# Coluna de rotura 
dfNinjasLong["Rotura_Linear"] = np.where(dfNinjasLong.NinjaInfo == 0, 1, 0)

# Coluna de Roturas Consecutivas cheia de zeros
dfNinjasLong["Rotura_Consecutiva"] = 0

# Criar a máscar para depois aplicar a função nos grupos estabelecidos
mask = ((dfNinjasLong["Rotura_Linear"] == 1) &
        (dfNinjasLong.groupby(["FDS", "STORE", "DESC_ARTIGO"])["Rotura_Linear"].shift(1) == 1))

# Usar a função soma para os grupos da máscara
dfNinjasLong["Rotura_Consecutiva"] = mask.groupby([dfNinjasLong["FDS"], dfNinjasLong["STORE"], dfNinjasLong["DESC_ARTIGO"]]).cumsum()

In [32]:
import requests
import pandas as pd
from io import StringIO

ids = pd.read_excel("D:\\B&N Dados\\Delta\\Padrão\\GoChill\\idsGoChill.xlsx")
ids = ids.ID.tolist()

url = []
for i in ids:
    url.append(f"https://app.brandsandninjas.com/answer_photos.csv?id={i}")

dataframes = []
for i in url:
    # Send an HTTP GET request to the URL
    response = requests.get(i)  # Use the current URL 'i' here

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the content of the response
        file_content = response.content

        # Create a Pandas DataFrame from the CSV content
        df = pd.read_csv(StringIO(file_content.decode('utf-8')))
        
        dataframes.append(df)
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")


dfFotos = pd.concat(dataframes, ignore_index=True)
dfFotos = dfFotos.rename(columns = {"photo_taken_at":"DATA", "store_name":"STORE_NAME"})
dfFotos['Hora_da_Foto'] = dfFotos['DATA'].str.split(' ').str[1]
dfFotos['DATA_Foto'] = dfFotos['DATA']
dfFotos['DATA'] = dfFotos['DATA'].str.split(' ').str[0]
dfFotos['DATA'] = pd.to_datetime(dfFotos['DATA'], format='%Y-%m-%d')

dicionárioLatLong = dict(zip(dfFotos['STORE_NAME'], dfFotos['location']))


dfFotos["photo_url2"] = dfFotos["photo_url"]

In [34]:
dicionarioCoordenadas = dict(zip(dfFotos['STORE_NAME'], dfFotos['location']))
dfNinjasLong["STORE_NAME2"] = dfNinjasLong["STORE_NAME"]
dfNinjasLong["Color"] = dfNinjasLong["Rotura_Linear"].map({0:"Produto Presente",1:"Produto em Rotura"})
dfNinjasLong["LatLong"] = dfNinjasLong["STORE_NAME"].map(dicionarioCoordenadas)
dfNinjasLong["Percentagem_de_Presença"] = dfNinjasLong["Num_Produtos"]/len(produtos)
id_Dia_da_Semana = {'Segunda': 1,
                    'Terça': 2,
                    'Quarta': 3,
                    'Quinta': 4,
                    'Sexta': 5,
                    'Sábado': 6,
                    'Domingo': 7}


dfNinjasLong["Order_Dia"] = dfNinjasLong["Dia_da_Semana"].map(id_Dia_da_Semana)

In [35]:
dfNinjasLong['DATA'] = pd.to_datetime(dfNinjasLong['DATA'], format='%Y-%m-%d')
dfNinjasLong['DATA'] = dfNinjasLong['DATA'].dt.strftime("%Y-%m-%d")
dfNinjasLong['DATA'] = pd.to_datetime(dfNinjasLong['DATA'], format='%Y-%m-%d')


In [36]:
dfFotos2 =  pd.merge(dfNinjasLong, dfFotos, how="left", on = ["DATA","STORE_NAME"]) 
dfFotos2 = dfFotos2.sort_values(by=["STORE","DESC_ARTIGO","DATA"])

In [37]:
dicionarioCodLojas = {v: k for k, v in dicionarioLojas.items()}

In [38]:
dfFotos2["STORE"] = dfFotos2["STORE_NAME"].map(dicionarioCodLojas) 

In [39]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor2.xlsx", sheet_name = "Lojas Sonae para o desafio")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjasLong = pd.merge(dfNinjasLong, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

In [40]:
dfFotos2 = pd.merge(dfFotos2, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

In [43]:
import clickhouse_connect
from clickhouse_driver import Client
from unidecode import unidecode

client = clickhouse_connect.get_client(host='ch.brandsandninjas.com', 
                                       port=443, 
                                       username='chninja', 
                                       password='ku43ueqnB5Q0AYb2C4FsJRTc7qX',
                                       database = "Delta")

qualbase = 2

if qualbase == 1:
    baseCH = dfNinjasLong.copy()
    tabela = "Delta_SemFotos" # GoChill4SemFotos2

if qualbase == 2:
    baseCH = dfFotos2.copy()
    tabela = "Delta_ComFotos" # GoChill4ComFotos


In [44]:
## Algumas alterações
baseCH['DATA']= pd.to_datetime(baseCH['DATA'], format='%Y-%m-%d')  # Passar a Data para datetime

baseCH.columns = [unidecode(col) for col in baseCH.columns]        # Tirar acentos e afins dos nomes das colunas porque 
                                                                   # o Clickhouse não gosta


## Listas para definir os tipos de dados de cada coluna a inserir

data = ["DATA"]
#data = [col for col in baseCH.columns if baseCH[col].dtype == 'datetime64[ns]']
texto = [col for col in baseCH.columns if baseCH[col].dtype == 'object']
inteiros = [col for col in baseCH.columns if baseCH[col].dtype == 'int64' or baseCH[col].dtype == 'int32']
floats = [col for col in baseCH.columns if baseCH[col].dtype == 'float64']


## Mudar inteiros para floats porque senão não pode haver missing values
for col_name in inteiros:
    baseCH[col_name] = baseCH[col_name].astype(float)

## Missing values em strings também estragam tudo
baseCH[texto] = baseCH[texto].fillna("-")
baseCH[texto] = baseCH[texto].astype(str)

#Função que vai fazer a schema
def schema(lista, tipo):
    result_list = [f"{element} {tipo}" for element in lista]
    return result_list

# Schema a ser feito
data1 = schema(data, "Date")
texto1 = schema(texto, "String")
inteiros1 = schema(inteiros, "Float64")
floats1 = schema(floats, "Float64")
total = tuple(data1 + texto1 + inteiros1 + floats1)
schema = ', '.join([column.replace("'", "") for column in total])

# Split the input string by commas
parts = schema.split(', ')
# Process each part and wrap the first word in double quotes
output_parts = []
for part in parts:
    words = part.split()
    if words:
        first_word = words[0]
        remaining_words = ' '.join(words[1:])
        output_part = f'"{first_word}" {remaining_words}'
        output_parts.append(output_part)
# Join the modified parts back into a string
schema = ', '.join(output_parts)

# Eliminar tabela que possa existir no CH com o mesmo nome
client.command(f'DROP TABLE IF EXISTS {tabela}')

# Criar tabela no CH
client.command(f'''
    CREATE TABLE IF NOT EXISTS {tabela} (
        {schema}
        ) ENGINE = MergeTree
        ORDER BY (DATA)
''')


# Exportar os dados para o clickhouse
client.insert_df(tabela, baseCH, column_names=baseCH.columns.tolist())